In [ ]:
%matplotlib inline
# import
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import json
import os
import pathlib
from matplotlib.colors import ListedColormap

#path to net analyis of each dataset
net_birth = "../../../eval-results/_server-results/net-BP-birth/net.eval" 
net_uni = "../../../eval-results/_server-results/net-BP-uni/net.eval" 
net_sap = "../../../eval-results/_server-results/net-BP-sap/net.eval" 

#path to bp results
bp_birth = "../../../eval-results/_server-results/test13_ppp/"
bp_uni = "../../../eval-results/_server-results/test17_ppp/"
bp_sap = "../../../eval-results/_server-results/test18_ppp/"

#path to alpha results
alpha_birth = "../../../eval-results/_server-results/test22_ppp/"
alpha_uni = "../../../eval-results/_server-results/test21_ppp/"
alpha_sap = "../../../eval-results/_server-results/test23_ppp/"

#path to bpp results
bpp_birth = "../../../eval-results/_server-results/test26_ppp/"
bpp_uni = "../../../eval-results/_server-results/test24_ppp/"
bpp_sap = "../../../eval-results/_server-results/test25_ppp/"

In [ ]:
def net1adder(row):
    #get net names
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].split('-')[0].replace(".pnml","")
    else:
        return 0
def net2adder(row):        
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].split('-')[1].split('.')[0].replace(".pnml","")
    else: 
        return 0
    
def pnml_remover(row):
    #get net names
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].replace(".pnml","").replace(".rdf","")
    else:
        return row['Name']

def net1NonTaus(row):
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return df_nets.at[row['net1'],'nNonSilentTransitions'] 
    else:
        return 0
def net2NonTaus(row):
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return df_nets.at[row['net2'],'nNonSilentTransitions'] 
    else:
        return 0
def sumNonTaus(row):    
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['net1NonTaus'] + row['net2NonTaus']
    else:
        return 0

def fetch_data(folder, net_analysis, dataset, profile):
    dir_list = next(os.walk(folder))[1]
    dfs = []
    #merge all 
    for subfolder in dir_list:
        #print(subfolder)
        evalFile = folder +"/" + subfolder +"/aggRetrospectiveResults.eval"
        confFile = folder +"/" + subfolder +"/config.log"
        if os.path.exists(evalFile):
            df = pd.read_csv(evalFile ,encoding="ISO-8859-1", skipinitialspace=True)
            with open(confFile) as json_file:
                conf = json.load(json_file)
                #add config information to dataframe 
                df['matcher'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - sim-weight=" + str(conf['matcher']['sim-weight']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh'] ) + " - Node Limit: " +str(conf['matcher']['ilp-node-limit']) + "- Time Limit : " +str(conf['matcher']['ilp-time-limit'])
                df['matcher_wo_weight'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh']) + " - Node Limit: " +str(conf['matcher']['ilp-node-limit']) + "- Time Limit : " +str(conf['matcher']['ilp-time-limit'])
                df['matcher_wo_limit'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - sim-weight=" + str(conf['matcher']['sim-weight']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh'] )
                df['complex-matches'] = conf['matcher']['complex matches']
                df['profile'] = conf['matcher']['profile']
                df['ilp'] =  conf['matcher']['ilp']
                df['word-sim'] =  conf['matcher']['word-sim']
                df['sim-weight'] = conf['matcher']['sim-weight']
                df['matcher-postprocessing-threshold'] = conf['matcher']['postprocessing-thresh']
                df['eval-postprocessing-threshold'] = conf['evaluation']['postprocessing-thresh']
                df['ILP-time-limit'] = conf['matcher']['ilp-time-limit']
                df['ILP-node-limit'] = conf['matcher']['ilp-node-limit']
            
            #df.set_index(['Name','matcher'])
            dfs.append(df)

    df_combined = pd.concat(dfs)

    #convert time
    df_combined['OVERALL TIME'] = df_combined['OVERALL TIME'].map(lambda x: x / 1000000000.)
    df_combined['BP TIME'] = df_combined['BP TIME'].map(lambda x: x / 1000000000.)
    df_combined['LABEL-SIM TIME'] = df_combined['LABEL-SIM TIME'].map(lambda x: x / 1000000000.)
    df_combined['LP TIME'] = df_combined['LP TIME'].map(lambda x: x / 1000000000.)
    df_combined['Dataset'] = dataset
    df_combined['Profile'] = profile

    #extend with net information stored in net_analysis
    df_nets = pd.read_csv(net_analysis ,encoding="ISO-8859-1", skipinitialspace=True)
    #df_nets.set_index('Name')


    df_combined['net1'] = df_combined.apply(lambda row: net1adder(row), axis=1)
    df_combined['net2'] = df_combined.apply(lambda row: net2adder(row), axis=1)
    #df_combined['net1NonTaus'] = df_combined.apply(lambda row: net1NonTaus(row), axis=1)
    #df_combined['net2NonTaus'] = df_combined.apply(lambda row: net2NonTaus(row), axis=1)
    #df_combined['sumNonTaus'] = df_combined.apply(lambda row: sumNonTaus(row), axis=1)
    #df_combined['Name'] = df_combined.apply(lambda row: pnml_remover(row), axis=1)
    return df_combined

In [ ]:
birth = fetch_data(bp_birth, net_birth ,"Birth", "BP")
birth = birth.append(fetch_data(alpha_birth, net_birth ,"Birth", "Alpha"))
birth = birth.append(fetch_data(bpp_birth, net_birth ,"Birth", "BP+"))

uni = fetch_data(bp_uni, net_uni, "Uni", "BP")
uni = uni.append(fetch_data(alpha_uni, net_uni, "Uni", "Alpha"))
uni = uni.append(fetch_data(bpp_uni, net_uni, "Uni", "BP+"))

sap = fetch_data(bp_sap, net_sap, "Sap", "BP")
sap = sap.append(fetch_data(alpha_sap, net_sap, "Sap", "Alpha"))
sap = sap.append(fetch_data(bpp_sap, net_sap, "Sap", "BP+"))

#concat and get micro scores only
birth = birth[birth.Name == "Aggregated (MICRO)"]
uni = uni[uni.Name == "Aggregated (MICRO)"]
sap = sap[sap.Name == "Aggregated (MICRO)"]

print("BEST:")
#sort 
birth = birth.sort_values(by="FSCORE", ascending=False)
print("Birth")
print(birth[["PRECISION","RECALL","FSCORE","sim-weight","eval-postprocessing-threshold","Profile"]][:10])
uni = uni.sort_values(by="FSCORE", ascending=False)
print("Uni")
print(uni[["PRECISION","RECALL","FSCORE","sim-weight","eval-postprocessing-threshold","Profile"]][:10])
sap = sap.sort_values(by="FSCORE", ascending=False)
print("SAP")
print(sap[["PRECISION","RECALL","FSCORE","sim-weight","eval-postprocessing-threshold","Profile"]][:10])

print("WORST:")
birth = birth.sort_values(by="FSCORE", ascending=True)
print("Birth")
print(birth[["PRECISION","RECALL","FSCORE","sim-weight","eval-postprocessing-threshold","Profile"]][:10])
uni = uni.sort_values(by="FSCORE", ascending=True)
print("Uni")
print(uni[["PRECISION","RECALL","FSCORE","sim-weight","eval-postprocessing-threshold","Profile"]][:10])
sap = sap.sort_values(by="FSCORE", ascending=True)
print("SAP")
print(sap[["PRECISION","RECALL","FSCORE","sim-weight","eval-postprocessing-threshold","Profile"]][:10])




In [ ]:
# What is the best and worst result (micro fscore) of a given folder
folder = "../../../eval-results/_server-results/test42_ppp/"
dir_list = next(os.walk(folder))[1]
min_path = ""
max_path = ""
maximum = 0.0
minimum = 1.0

    #merge all 
for subfolder in dir_list:
    evalFile = folder +"/" + subfolder +"/aggRetrospectiveResults.eval"
    if os.path.exists(evalFile):
        df = pd.read_csv(evalFile ,encoding="ISO-8859-1", skipinitialspace=True)
        df = df[df.Name == "Aggregated (MICRO)"]
        if df.iloc[0].FSCORE > maximum:
            max_path = evalFile
            maximum = df.iloc[0].FSCORE
            max_col = df.iloc[0]
        if df.iloc[0].FSCORE < minimum:
            min_path = evalFile
            minimum = df.iloc[0].FSCORE
            min_col = df.iloc[0]
        

print("Maximum: " + str(maximum) + " - "+ max_path)
#print(max_col)
print("Minimum: " + str(minimum) +" - "+ min_path)
#print(min_col)


